## Preprocess the data

In [1]:
# import torch
# torch.__version__
# torch.version.cuda

In [2]:
# ! pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
# ! pip install --upgrade --force-reinstall torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu110.html
# ! pip install --upgrade --force-reinstall torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu110.html
# ! pip install --upgrade --force-reinstall torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu110.html
# ! pip install --upgrade --force-reinstall torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu110.html
# ! pip install torch-geometric
# ! pip install numpy==1.18.0

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [4]:
import time
import pickle
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torch.nn import Sequential as Seq, Linear, ReLU

from torch.utils.tensorboard import SummaryWriter

from collections import Counter
from tqdm.notebook import tqdm

from sklearn import metrics
from sklearn.metrics import classification_report, f1_score

In [5]:
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from torch_geometric.data import DataLoader, InMemoryDataset, Data

from torch_geometric.utils import remove_self_loops, add_self_loops

In [6]:
device = 'cuda'

# Dataset

In [8]:
conll_data = pickle.load(open('conll_graph_all.pickle', 'rb'))
vocabulary = pickle.load(open('vocabulary_all.pickle', 'rb'))
voc2id = {key:{l: i for i, l in enumerate(vocabulary[key])} for key in vocabulary}
labels = pickle.load(open('labels.pickle', 'rb'))
label2id = {l: i for i, l in enumerate(labels)}

In [12]:
class CoNLLDataset(InMemoryDataset):
    def __init__(self, data, split, voc2id, labels, window_size=3, root='.', transform=None, pre_transform=None):
        self.dataset = data[split]
        self.voc2id = voc2id
        self.labels = labels
        self.label2id = {l: i for i, l in enumerate(labels)}
        
        self.window_size = window_size
        self.split = split
        
        super(CoNLLDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

        
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return ['/data/graphner_embeddings/node_selfloop_simple_features/'+self.split]

    def download(self):
        pass
    
    def process(self):
        data_list = []        
    
        for doc_i, doc in tqdm(enumerate(self.dataset), total=len(self.dataset)):
            features = ['word', 'pos', 'chunk', 'extra'] # to keep the same order

            word_id  = [self.voc2id['word'][doc['word'][0]]]
            len_so_far = len(self.voc2id['word'])
            pos_id   = [len_so_far + self.voc2id['pos'][doc['pos'][0]]]
            len_so_far += len(self.voc2id['pos']) 
            chunk_id = [len_so_far + self.voc2id['chunk'][doc['chunk'][0]]]
            len_so_far += len(self.voc2id['chunk'])

            extra_ids = []
            for v in doc['extra']:
                extra_ids.append(len_so_far + self.voc2id['extra'][v])
            len_so_far += len(self.voc2id['extra'])

            window_size = max(len(doc['left_context']), len(doc['right_context'])) if self.window_size == 'all' \
                        else self.window_size

            ids = word_id + pos_id + chunk_id + extra_ids
            edges = [(i+1, 0) for i, n in enumerate(ids[1:])]

            context_word_ids = []
            for window  in [doc['left_context'][-window_size:], doc['right_context'][:window_size]]:
                for i, w in enumerate(window):

                    # All nodes relate to the central node
                    # edges.append((len(ids), 0)) 

                    # Nodes linked by order in the sentence
                    edges.append((len(ids), 0 if i == 0 else (len(ids) - 1)))
                    ids.append(self.voc2id['word'][w])
            
            # add self loop
            edges.append((0, 0))

            x = torch.LongTensor(ids).unsqueeze(1)
            y = torch.tensor([self.label2id[doc['label']]])
            edge_index = torch.tensor(list(zip(*edges)))

            try:
                data = Data(x=x, y=y, edge_index=edge_index)
                data_list.append(data)
            except:
                if len(edge_index) == 0:
                    continue
                print(x, y, edge_index)
                break

        data, slices = self.collate(data_list)
        torch.save((data, slices), '/data/graphner_embeddings/node_selfloop_simple_features/'+self.split)

In [14]:
train_dataset = CoNLLDataset(data=conll_data, split='train', voc2id=voc2id, labels=labels, window_size='all')
val_dataset = CoNLLDataset(data=conll_data, split='validation', voc2id=voc2id, labels=labels, window_size='all')
test_dataset = CoNLLDataset(data=conll_data, split='test', voc2id=voc2id, labels=labels, window_size=('all'))

Processing...



Done!
Processing...



Done!
Processing...



Done!


# Model

In [15]:
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]

        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [16]:
embed_dim = 512
dim_input = sum(len(voc2id[k]) for k in ['word', 'pos', 'chunk', 'extra', 'classes'])
n_labels = len(labels)
dropout_rate = 0.5

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, embed_dim)
        self.pool1 = TopKPooling(embed_dim, ratio=0.8)
        self.conv2 = SAGEConv(embed_dim, embed_dim)
        self.pool2 = TopKPooling(embed_dim, ratio=0.8)
        self.conv3 = SAGEConv(embed_dim, embed_dim)
        self.pool3 = TopKPooling(embed_dim, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=dim_input, embedding_dim=embed_dim)
        # self.lin0 = torch.nn.Linear(dim_input, embed_dim)
        self.lin1 = torch.nn.Linear(embed_dim*2, embed_dim)
        self.lin2 = torch.nn.Linear(embed_dim, embed_dim//2)
        self.lin3 = torch.nn.Linear(embed_dim//2, n_labels)
        self.bn1 = torch.nn.BatchNorm1d(embed_dim)
        self.bn2 = torch.nn.BatchNorm1d(embed_dim //2)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # x = self.lin0(x.float())
        x = self.item_embedding(x)
        x = x.squeeze(1)

        x = F.relu(self.conv1(x, edge_index))
        
        z = self.pool1(x, edge_index, None, batch)
        x, edge_index, _, batch, _, _ = z
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=dropout_rate, training=self.training)

        # x = torch.sigmoid(self.lin3(x))
        x = self.lin3(x).squeeze(1)
        # print(x)
        
        return x

In [20]:
model = Net().to(device)

writer = SummaryWriter(comment='gcn-justids-winall-wei2-lr1e3-mom0.9-wd5e4-embdim512-dr0.5-bs64-notall', log_dir=None,)
logs = []

In [21]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [29]:
optimizer_params = {'lr': 5e-4,
                    'momentum': 0.9, 
                    'weight_decay': 5e-4,
                   }

device = torch.device('cuda')
weights1 = [0.5530, 1.0000, 0.0317, 0.4590, 0.4120]
weights2 = [0.7436, 1.0000, 0.1780, 0.6775, 0.6419]

weights = torch.Tensor(weights2).to(device)

optimizer = torch.optim.SGD(model.parameters(),  **optimizer_params)
loss_fn = torch.nn.CrossEntropyLoss(weight=weights)
# print()#weights)

In [30]:
torch.cuda.device_count()

2

In [31]:
def train(loader):
    model.train()

    loss_all = 0
    print_loss_every = int(len(loader) / 5)
    for ii, data in enumerate(tqdm(loader, total=len(loader))):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = loss_fn(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        if ii % print_loss_every == 0:
            print('== Loss at', ii, ' : ', data.num_graphs * loss.item())

    return loss_all / len(train_dataset)

In [32]:
def evaluate(loader, split=''):
    print('==== Evaluation on the', split.upper(), ' set ====')
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in tqdm(loader, total=len(loader)):

            data = data.to(device)
            pred = torch.nn.Softmax(dim=1)(model(data)).detach().cpu().numpy()
            label = data.y.detach().cpu().numpy()
            
            predictions.append(np.argmax(pred, axis=1))
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    print(classification_report(predictions, labels, digits=4))
    
    micro_F1 = metrics.f1_score(labels, predictions, average='micro')
    macro_F1 = metrics.f1_score(labels, predictions, average='macro')
    weighted_F1 = metrics.f1_score(labels, predictions, average='weighted')

        
    return (micro_F1, macro_F1, weighted_F1)

In [33]:
for epoch in range(5):
    loss = train(train_loader)
    val_acc = evaluate(val_loader, 'val')
    logs.append((loss, val_acc))
    
    writer.add_scalar("Learning_rate", optimizer_params['lr'], len(logs))
    writer.add_scalar("Loss/train", loss, len(logs))
    writer.add_scalar("micro_F1/dev", val_acc[0], len(logs))
    writer.add_scalar("macro_F1/dev", val_acc[1], len(logs))
    writer.add_scalar("weighted_F1/dev", val_acc[2], len(logs))

    print(f'Epoch: {len(logs):03d}, Loss: {loss:.5f}')
    # print('train_acc', train_acc)
    print('val_acc', val_acc)

== Loss at 0  :  0.2786133587360382
== Loss at 558  :  4.247607707977295
== Loss at 1116  :  3.154083490371704
== Loss at 1674  :  0.36151719093322754
== Loss at 2232  :  0.08238252997398376
== Loss at 2790  :  0.45689702965319157

==== Evaluation on the VAL  set ====



              precision    recall  f1-score   support

           0     0.8658    0.8596    0.8627      2109
           1     0.7695    0.8471    0.8065      1151
           2     0.9906    0.9878    0.9892     36442
           3     0.7393    0.8275    0.7810      1844
           4     0.9050    0.8465    0.8747      3354

    accuracy                         0.9610     44900
   macro avg     0.8541    0.8737    0.8628     44900
weighted avg     0.9624    0.9610    0.9615     44900

Epoch: 031, Loss: 0.02005
val_acc (0.9610467706013363, 0.8628216623803944, 0.9605982063681884)


== Loss at 0  :  0.8660431504249573
== Loss at 558  :  0.21276211738586426
== Loss at 1116  :  0.5954366326332092
== Loss at 1674  :  0.27227213978767395
== Loss at 2232  :  0.4164305031299591
== Loss at 2790  :  0.05886933649890125

==== Evaluation on the VAL  set ====



              precision    recall  f1-score   support

           0     0.8739    0.8527    0.8632      2146
           1     0.7893    0.8143    0.8016      1228
           2     0.9916    0.9870    0.9893     36508
           3     0.7393    0.8191    0.7772      1863
           4     0.8805    0.8754    0.8779      3155

    accuracy                         0.9610     44900
   macro avg     0.8549    0.8697    0.8618     44900
weighted avg     0.9621    0.9610    0.9615     44900

Epoch: 032, Loss: 0.01742
val_acc (0.9610244988864143, 0.8618399241438386, 0.9605636042067732)


== Loss at 0  :  0.24596163630485535
== Loss at 558  :  0.42124319076538086
== Loss at 1116  :  0.49599549174308777
== Loss at 1674  :  0.8637933135032654
== Loss at 2232  :  0.78936767578125
== Loss at 2790  :  5.205151438713074

==== Evaluation on the VAL  set ====



              precision    recall  f1-score   support

           0     0.8314    0.9049    0.8666      1924
           1     0.7735    0.8529    0.8113      1149
           2     0.9938    0.9850    0.9894     36662
           3     0.7674    0.7608    0.7641      2082
           4     0.8677    0.8829    0.8752      3083

    accuracy                         0.9608     44900
   macro avg     0.8468    0.8773    0.8613     44900
weighted avg     0.9620    0.9608    0.9613     44900

Epoch: 033, Loss: 0.01625
val_acc (0.9607795100222717, 0.8613159165799864, 0.9602895738835723)


== Loss at 0  :  0.16661709547042847
== Loss at 558  :  0.42587175965309143
== Loss at 1116  :  0.22452785074710846
== Loss at 1674  :  0.695753276348114
== Loss at 2232  :  0.7455546855926514
== Loss at 2790  :  0.11782485526055098

==== Evaluation on the VAL  set ====



              precision    recall  f1-score   support

           0     0.8481    0.8898    0.8685      1996
           1     0.7751    0.8422    0.8072      1166
           2     0.9926    0.9868    0.9897     36552
           3     0.7786    0.7602    0.7693      2114
           4     0.8677    0.8861    0.8768      3072

    accuracy                         0.9612     44900
   macro avg     0.8524    0.8730    0.8623     44900
weighted avg     0.9619    0.9612    0.9615     44900

Epoch: 034, Loss: 0.01559
val_acc (0.9611581291759466, 0.8622871911018233, 0.9608616433350432)


== Loss at 0  :  0.6512431502342224
== Loss at 558  :  0.12677499651908875
== Loss at 1116  :  0.28368666768074036
== Loss at 1674  :  0.7867997288703918
== Loss at 2232  :  0.9612324237823486
== Loss at 2790  :  0.7040765602141619

==== Evaluation on the VAL  set ====



              precision    recall  f1-score   support

           0     0.8596    0.8742    0.8668      2059
           1     0.7790    0.8386    0.8077      1177
           2     0.9913    0.9876    0.9895     36474
           3     0.7611    0.7967    0.7785      1972
           4     0.8878    0.8654    0.8765      3218

    accuracy                         0.9614     44900
   macro avg     0.8558    0.8725    0.8638     44900
weighted avg     0.9622    0.9614    0.9617     44900

Epoch: 035, Loss: 0.01483
val_acc (0.9613808463251671, 0.8637968167932218, 0.9610366312837183)


In [34]:
test_acc = evaluate(test_loader, 'test')
print('test_acc', test_acc)

==== Evaluation on the TEST  set ====



              precision    recall  f1-score   support

           0     0.8156    0.8173    0.8164      1921
           1     0.7157    0.7057    0.7107       931
           2     0.9810    0.9846    0.9828     32531
           3     0.7264    0.6844    0.7048      2646
           4     0.7714    0.7832    0.7773      2731

    accuracy                         0.9374     40760
   macro avg     0.8020    0.7950    0.7984     40760
weighted avg     0.9366    0.9374    0.9369     40760

test_acc (0.9373650637880275, 0.7983866027058582, 0.9378161293910184)
